## Tranformar datase de IDS para utilizar apenas dimensão economica

In [ ]:
import pandas as pd
from lxml import html
import requests
import zipfile
import io
import os
from pathlib import Path
import csv

In [ ]:
df = pd.read_csv("http://ftp.dadosabertos.ans.gov.br/FTP/PDA/historico_idss/historico_idss.csv",sep=";",encoding="ISO-8859-1")

In [ ]:
df.columns

In [ ]:
df = df[['Registro','Razao_Social','Modalidade','IDSM_2018_2017']]

## Filtrando para obter apenas cooperativas

In [ ]:
df_cooperativas = df[df['Modalidade'] == "Cooperativa Médica"]

In [ ]:
len(df_cooperativas)

In [ ]:
df_cooperativas.dropna(inplace=True) 

In [ ]:
len(df_cooperativas)

In [ ]:
df_cooperativas.head()

In [ ]:
def converter_para_float(x):
    try:
        return float(x.replace(".","").replace(",","."))
    except:
        return 0

### Tranformando índices em variáveis do tipo float

In [ ]:
colunas_indices = ['IDSM_2018_2017']
for coluna in colunas_indices:
    df_cooperativas[coluna] = df_cooperativas[coluna].apply(lambda x: converter_para_float(x))

In [ ]:
df_cooperativas.head()

## Filtrando para pegar as cooperativas com o IDMS igual a 1

In [ ]:
df_cooperativas = df_cooperativas[df_cooperativas['IDSM_2018_2017']==1]

In [ ]:
# df_cooperativas.set_index("Registro",inplace=True)

In [ ]:
df_porte = pd.read_csv("../entrada/porte_operadoras.csv",encoding="ISO-8859-1")
df_porte.drop(columns=['beneficiarios'],inplace=True)
df_porte = df_porte[(df_porte['ano'] == 2019) & (df_porte['trimestre'] == 3)]

In [ ]:
df_cooperativas_indice_um =  df_cooperativas.merge(df_porte,left_on="Registro",right_on="Registro_ANS")

In [ ]:
df_cooperativas_indice_um['porte'].value_counts()

In [ ]:
df_porte = df_cooperativas_indice_um[['Registro_ANS',"porte"]]

In [ ]:
df_porte.to_csv("../Datasets/idss_alto_nivel.csv",index=False,encoding="ISO-8859-1")